In [ ]:
import pandas as pd
df = pd.read_csv("anime.csv")
df

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [ ]:
missing_values = df.isnull().sum()
missing_values

anime_id      0
name          0
genre        62
type         25
episodes      0
rating      230
members       0
dtype: int64

In [ ]:
# Filling of missing values
# non-numeric = unknown & numeric values = avg values

df['genre'].fillna('Unknown', inplace=True)
df['type'].fillna('Unknown', inplace=True)

avg_rating = df["rating"].mean()
df['rating'].fillna(avg_rating, inplace=True)


In [ ]:
# verification of missing values
missing_values_after = df.isnull().sum()
missing_values_after

anime_id    0
name        0
genre       0
type        0
episodes    0
rating      0
members     0
dtype: int64

In [ ]:
#Normalisation of rating and converting catagorical numric (genre)
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler

genre_encoded = df['genre'].str.get_dummies(sep=', ')

scaler = StandardScaler()
rating_normalized = scaler.fit_transform(df[['rating']])

# Combine into a single DataFrame
features = pd.concat([genre_encoded, pd.DataFrame(rating_normalized, columns=['rating_normalized'])])
features.head()

,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Ecchi,Fantasy,Game,...,Space,Sports,Super Power,Supernatural,Thriller,Unknown,Vampire,Yaoi,Yuri,rating_normalized
0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NaN
1,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
2,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,NaN
4,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
genre_encoded = df['genre'].str.get_dummies(sep=', ')

In [ ]:
# Normalising of rating.
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
rating_normalized = scaler.fit_transform(df[['rating']])

In [ ]:
# combining features
features = pd.concat([genre_encoded, pd.DataFrame(rating_normalized, columns=['rating_normalized'])], axis=1)

In [ ]:
# Computing Cosine Similarity
from sklearn.metrics.pairwise import cosine_similarity
similarity_matrix = cosine_similarity(features)

In [ ]:
# Recommendation Function
def recommend_anime(anime_title, similarity_matrix, df, top_n=10):
    anime_idx = df[df['name'] == anime_title].index[0]
    similarity_scores = list(enumerate(similarity_matrix[anime_idx]))
    similarity_scores = sorted(similarity_scores, key=lambda x: x[1], reverse=True)
    similar_anime_indices = [i[0] for i in similarity_scores[1:top_n+1]]
    return df['name'].iloc[similar_anime_indices]

## Using The Function to recommend anime.

In [ ]:
recommendations = recommend_anime("Kimi no Na wa.", similarity_matrix, df, top_n=5)
print("Recommendations for 'Kimi no Na wa.':")
print(recommendations)

Recommendations for 'Kimi no Na wa.':
208                        Kokoro ga Sakebitagatterunda.
504    Clannad: After Story - Mou Hitotsu no Sekai, K...
60                                    Hotarubi no Mori e
10                                  Clannad: After Story
219    Yahari Ore no Seishun Love Comedy wa Machigatt...
Name: name, dtype: object


In [ ]:
recommendations = recommend_anime("Under World", similarity_matrix, df, top_n=5)
print("Recommendations for 'Under World':")
print(recommendations)

Recommendations for 'Under World':
12287                          Tenshi no Habataki Jun
12288                                The Satisfaction
12260                         Hokenshitsu de Aimashou
12289    Toushindai My Lover: Minami tai Mecha-Minami
12267                                   Lovely Series
Name: name, dtype: object
